# Splitting of tweets into xlsx-files

This notebook is to divide 520 tweets to eight xlsx-files, so that every tweet will be labelled by three persons. This means that each member of the group will get a excel-file that has 195 tweets.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import json
import re

#Imports for encrypting
import numpy as npv
import base64

from getpass import getpass
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

## Test and train sets

In [ ]:
#reads data from a file
data_filepath = r'C:\Users\Inka\Downloads\kws_final_01-04-2020.json'

data_arr = []
column_names = ['id_str','text']
with open(data_filepath, 'r') as f:
    for tweet in f:
        selected_row = []
        json_tweet = json.loads(tweet)
            # filter out retweets and non-English tweets:
            #Extended tweets
        try:
            if not json_tweet['retweeted'] and 'RT @' not in json_tweet['text'] and json_tweet['extended_tweet'] != False: #has extended
                tweet_id = json_tweet['id_str']
                text = json_tweet['extended_tweet']['full_text']
                new_list = [tweet_id, text]
                data_arr.append(new_list)
                #Not extended 
        except: 
            if not json_tweet['retweeted'] and 'RT @' not in json_tweet['text']:
                for col in column_names:
                    selected_row.append(json_tweet[col])
                data_arr.append(selected_row)

    data = pd.DataFrame(data_arr, columns=column_names)

In [ ]:
len(data)

### Filtering

In [ ]:
keywords = pd.read_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Keywords.xlsx')
keywords= keywords['keyword'].tolist()

In [ ]:
keywords

In [ ]:
rows_list = []
for i in range(len(data)):
    res = [ele for ele in keywords if(ele in data.iloc[i, 1])]
    if bool(res) == True:
        tweet_id = data.iloc[i, 0]
        text = data.iloc[i, 1]
        new_list = [tweet_id, text]
        rows_list.append(new_list)
        
new_data = pd.DataFrame(rows_list, columns=column_names)

In [ ]:
new_data

In [ ]:
#Each 8 members labels 195 tweets.
s = 520 #sample size
n = 3 #the amount that each tweet is labelled
p = 8 #number of people

In [ ]:
#Collects 520 random tweets for labelling from the final data
data_set, label_set = train_test_split(new_data, test_size = 520)

In [ ]:
label_set

In [ ]:
#Checks how many same tweets the previous set contains with the new set
og_set_1 = pd.read_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Original_set.xlsx')
og_set_1

count = 0
for i in range(520):
    for j in range(520):
        if og_set_1.iloc[i, 1] == label_set.iloc[j, 1]:
            print(label_set.iloc[j, 1])
            print(og_set_1.iloc[i, 1])
            count = count +1

print(count)

In [ ]:
print(label_set)
#Saves the original set:
label_set.to_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Original_set_2.xlsx', index = False)

In [ ]:
id_list = label_set['id_str'].tolist()
id_list[0]

## Pseudonymization

In [ ]:
def initialize_crypto(password):
    """
    Derives crypto key using the password and initializes the crypto library
    that can be then called via encrypt and decrypt functions.
    -----
    Returns: cryptography.fernet.Fernet object
    """
    password=password.encode()
    salt = b'm\xfffFvxfb\xbexB\x7f2\xaa\x1dj\x8c\x8f\xf1\\{' 
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )

    return Fernet(base64.urlsafe_b64encode(kdf.derive(password)))

In [ ]:
def encrypt_ids_base64(ids, password):
    """
    Encrypts a list of ids. Works on both strings and ints.
    -----
    Returns: list of bytes-typeobjects
    """
    
    crypto = initialize_crypto(password)
    #if isinstance(ids[0], int):
    return [base64.urlsafe_b64encode(crypto.encrypt(bytes(str(ID), 'utf-8'))) for ID in ids]
    #else:
    #    return [base64.urlsafe_b64encode(crypto.encrypt(bytes(ID, 'utf-8'))) for ID in ids] 

In [ ]:
def decrypt_ids_base64(encr_ids, password):
    """
    Decrypts a list of bytes objects..
    -----
    Returns: list of strings
    """
    
    crypto = initialize_crypto(password)
    keys = [crypto.decrypt(base64.b64decode(encr_ID.replace("b\'", '').replace("'",""))).decode("utf-8")
            for encr_ID in encr_ids] 
    #try:
    #    return [int(k) for k in keys]
    #except Exception as ex:
    return keys

### Insert password and encrypt

In [ ]:
password = getpass("Please enter a passphrase > : \n")    # Reads what user inputs

In [ ]:
#Encrypt
new_id_list = encrypt_ids_base64(id_list, password)

In [ ]:
new_id_list

### Decrypt

In [ ]:
#list = lb1['id_str'].tolist()

In [ ]:
T64 = decrypt_ids_base64(list, password)

In [ ]:
T64

## Dividing the data into xlsx-files

In [ ]:
label_set

In [ ]:
#Changes id-column to encrypted version
label_set = label_set.drop(columns=['id_str'])
label_set = np.c_[new_id_list, label_set]
label_set = pd.DataFrame(label_set, columns=['id_str', 'text'])

In [ ]:
label_set

In [ ]:
#adds extra column for labelling
label_set["label"] = "" #label_df

#Splits the new_df into number of p dataframes:
df_split = np.array_split(label_set, p)

df1 = df_split[0].append([df_split[1], df_split[2]])
df2 = df_split[3].append([df_split[4], df_split[5]])
df3 = df_split[6].append([df_split[7], df_split[0]])
df4 = df_split[1].append([df_split[2], df_split[3]])
df5 = df_split[4].append([df_split[5], df_split[6]])
df6 = df_split[7].append([df_split[0], df_split[1]])
df7 = df_split[2].append([df_split[3], df_split[4]])
df8 = df_split[5].append([df_split[6], df_split[7]])

In [ ]:
df1.to_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Inka_2.xlsx', index = False)
df2.to_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Maria_2.xlsx', index = False)
df3.to_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Ville_2.xlsx', index = False)
df4.to_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Maryam_2.xlsx', index = False)
df5.to_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Annika_2.xlsx', index = False)
df6.to_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Veera_2.xlsx', index = False)
df7.to_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Nuutti_2.xlsx', index = False)
df8.to_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Estanislao_2.xlsx', index = False)

In [ ]:
#Let's read the exels
lb1 = pd.read_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Inka_Finished.xlsx')
lb2 = pd.read_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Maria_Finished.xlsx')
lb3 = pd.read_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Ville_Finished.xlsx')
lb4 = pd.read_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Maryam_Finished.xlsx')
lb5 = pd.read_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Annika_Finished.xlsx')
lb6 = pd.read_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Veera_Finished.xlsx')
lb7 = pd.read_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Nuutti_Finished.xlsx')
lb8 = pd.read_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Labelling_Estanislao_Finished.xlsx')

In [ ]:
lb1.head()

## Combining the xlsx-files into one

In [ ]:
#Combines all the files together:
combined = lb1.append([lb2, lb3, lb4, lb5, lb6, lb7, lb8])
combined.reset_index(inplace=True)
combined = combined.drop(columns=['index']).drop(columns=['Unnamed: 3']).drop(columns=['Unnamed: 4'])

In [ ]:
combined

In [ ]:
label_list = []
count = 0

for i in range(s):

    label1 = combined.iloc[i, 1]
    label2 = combined.iloc[i + s, 1] 
    label3 = combined.iloc[i + s*2, 1]
    
    if label1 == label2:
        label = label1
        label_list.append(label)
    elif label2 == label3:
        label = label2
        label_list.append(label)
    elif label3 == label1:
        label = label3
        label_list.append(label)
    elif label1 != label2 and label2 != label3:
        count = count + 1
        print(combined.iloc[i, 2])
        print('-------------------------------')
        label = 4
        label_list.append(label)
    else:
        label = 'xxx'
        juu = juu + 1
        label_list.append(label)
        print(combined.iloc[i, 2])


In [ ]:
count

In [ ]:
print(len(label_list))

numb1 = 0
numb2 = 0
numb3 = 0
numb4 = 0

for i in range(520):
    if label_list[i] == 1:
        numb1 = numb1 +1
    elif label_list[i] == 2:
        numb2 = numb2 +1
    elif label_list[i] == 3:
        numb3 = numb3 +1
    elif label_list[i] == 4:
        numb4 = numb4 +1
    else:
        numbx = numbx + 1
        
print("Label 1 amount:", numb1/520*100, "%")
print("Label 2 amount:", numb2/520*100, "%")
print("Label 3 amount:", numb3/520*100, "%")
print("Label 4 amount:", numb4/520*100, "%")

### Merging the labels with the tweets

In [ ]:
#With original ids:
#orig_set = pd.read_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Original_set.xlsx')
#final_set = np.c_[orig_set, label_list]
#final_set = pd.DataFrame(final_set, columns=['id', 'text', 'label'])

In [ ]:
comb_520 = combined.head(520)
comb_520 = comb_520.drop(columns=['label'])

final_set = np.c_[comb_520, label_list]
final_set = pd.DataFrame(final_set, columns=['id_str', 'text', 'label'])

In [ ]:
final_set

In [ ]:
final_set.to_excel(r'C:\Users\Inka\Desktop\Koulu\Project course\Labelling\Final_set.xlsx', index = False)